In [7]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error
import warnings

warnings.filterwarnings('ignore')

# Load data
data = pd.read_csv('price.csv', sep=';')

# Forward fill missing date values
data['Tanggal'] = pd.to_datetime(data['Tanggal'])
data = data.set_index('Tanggal')
data = data.resample('D').ffill()

# Drop any remaining missing values
data = data.dropna()

# Normalize data
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

# Define train and test sets
train_size = int(len(data_scaled) * 0.8)
train_data, test_data = data_scaled[:train_size], data_scaled[train_size:]

# Define function to create LSTM model
def create_model(neurons, dropout_rate):
    model = Sequential()
    model.add(LSTM(neurons, return_sequences=True, input_shape=(train_data.shape[1], 1)))
    model.add(Dropout(dropout_rate))
    model.add(LSTM(neurons, return_sequences=True))
    model.add(Dropout(dropout_rate))
    model.add(LSTM(neurons))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model


# Define hyperparameters to tune
units = [256]
dropout_rates = [0.2, 0.4, 0.6]
epochs = 1000
batch_size = 64
learning_rate = 0.001

# Perform grid search to find optimal hyperparameters
best_mse = float('inf')
for unit in units:
    for rate in dropout_rates:
        model = create_model(unit, rate)
        es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
        history = model.fit(train_data, train_data[:, 0], epochs=epochs, batch_size=batch_size, verbose=1, validation_split=0.2, callbacks=[es])
        predictions = model.predict(test_data)
        mse = mean_squared_error(test_data[:, 0], predictions)
        print('Units:', unit, 'Dropout rate:', rate, 'MSE:', mse)
        if mse < best_mse:
            best_mse = mse
            best_params = {'Units': unit, 'Dropout rate': rate}

print('Best hyperparameters:', best_params, 'Best MSE:', best_mse)

Epoch 1/1000
32/47 [===================>..........] - ETA: 2s - loss: 0.0249

KeyboardInterrupt: 